### Section 1

In [ ]:
import pandas as pd
import numpy as np
import math
from scipy import stats
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression

In [ ]:
arrest = pd.read_csv("Arrest_Data_from_2010_to_Present.csv")

In [ ]:
arrest.info()

In [ ]:
arrest.describe()

In [ ]:
arrest.head(5)

In [ ]:
arrest['date'] = pd.to_datetime(arrest['Arrest Date'])
arrest.info()

In [ ]:
arrest_2018 = arrest[(arrest['date'] >= '01/01/2018') & (arrest['date'] < '01/01/2019')]
len(arrest_2018)

In [ ]:
arrest_sub_1 = arrest_2018[(arrest_2018['Charge Group Description'] == 'Vehicle Theft') | (arrest_2018['Charge Group Description'] == 'Robbery') | (arrest_2018['Charge Group Description'] == 'Burglary') | (arrest_2018['Charge Group Description'] == 'Receive Stolen Property')]

print(arrest_sub_1['Age'].describe())
print(np.quantile(arrest_sub_1['Age'], .75))
print(np.quantile(arrest_sub_1['Age'], .95))



In [ ]:
arrest_2018.groupby('Area ID')['Report ID'].count()

In [ ]:
#10951+7345+5927+3590+3875+9715+2467+2687+6193+3562+3452+6259+5427+6778+4947+3006+3146+3601+4357+3898+3094

In [ ]:
arrest_2018[arrest_2018['Area ID'] == 1]

In [ ]:
arrest_2018['Charge Group Description'].unique()

In [ ]:
arrest_sub_2 = arrest_2018[(arrest_2018['Charge Group Description'] != 'Non-Criminal Detention') & (arrest_2018['Charge Group Description'] != 'Pre-Delinquency')]

arrest_sub_2 = arrest_sub_2.dropna(subset=['Charge Group Description']) 

arrest_sub_2['Charge Group Description'].unique()

In [ ]:
sample_mean = arrest_sub_2.groupby('Charge Group Description')['Age'].mean()

In [ ]:
pop_mean = arrest_sub_2['Age'].mean()

In [ ]:
sample_sd = arrest_sub_2.groupby('Charge Group Description')['Age'].std()

In [ ]:
z = (sample_mean - pop_mean)/sample_sd

In [ ]:
abs(z).max()

In [ ]:
arrest['year'] = arrest['date'].dt.year
arrest_felony = arrest[arrest['Arrest Type Code'] == 'F']

In [ ]:
felony_2010_2018 = arrest_felony[(arrest_felony['date'] >= '01/01/2010') & (arrest_felony['date'] < '01/01/2019')]

counts = felony_2010_2018.groupby(['year'])['Report ID'].count().values.reshape(-1, 1)

In [ ]:
c = np.array(range(2010,2019)).reshape(-1, 1) 

In [ ]:
regressor = LinearRegression()  
regressor.fit(c, counts)
print(regressor.intercept_)
print(regressor.coef_)

print(regressor.intercept_ + regressor.coef_ * 2019)
count_pred = c * regressor.coef_ + regressor.intercept_

plt.plot(c, count_pred, '-r')
plt.scatter(c, counts, alpha=0.5)

In [ ]:
arrest_by_location = arrest.dropna(subset=['Location']) 
lat_lon = arrest_by_location['Location'].str.split(',', expand=True)

lats = lat_lon[0].str.replace('(','').values.reshape(-1)
lons = lat_lon[1].str.replace(')','')
lons = lons.str.replace(' ','')

In [ ]:
lats = lats.astype(float)
lons = lons.astype(float)

arrest_by_location['lats'] = lats
arrest_by_location['lons'] = lons
print(len(arrest_by_location))

arrest_by_location = arrest_by_location[arrest_by_location['lats'] != 0.0]
print(len(arrest_by_location))
arrest_by_location = arrest_by_location[arrest_by_location['lons'] != 0.0]
print(len(arrest_by_location))
arrest_by_location = arrest_by_location.reset_index(drop=True)

In [ ]:
def distance(lat1,lon1,lat2,lon2):
    R = 6371
    lat1 = (math.pi/180) * lat1
    lon1 = (math.pi/180) * lon1
    lat2 = (math.pi/180) * lat2
    lon2 = (math.pi/180) * lon2
    delta_lat = lat2 - lat1
    delta_lon = lon2 - lon1
    mean_lat = (lat1 + lat2)/2
    return R * math.sqrt(delta_lat ** 2 + (math.cos(mean_lat) * delta_lon) ** 2)

In [ ]:
distance(34.050536, -118.247861, 33.9543, -118.2827)

In [ ]:
lat1 = 34.050536
lon1 = -118.247861
s=[]
for i in range(0,len(arrest_by_location)+1):
    s.append(distance(lat1,lon1,arrest_by_location['lats'][i],arrest_by_location['lons'][i]))

In [ ]:
print(len(s))
s = np.sort(s)
print(len(s[s<=2.0]))

### Section 3

In [37]:
import pandas as pd
import numpy as np
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import plotly.express as px
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA

In [38]:
DataFileIn = open("may12pub.dat", "r")
DataList = DataFileIn.readlines()
DataList.sort()
DataFileIn.close()
DataTextOut = open('may12pub.txt', 'w')
DataTextOut.writelines(DataList) # Write a sequence of strings to a file
DataTextOut.close()

In [39]:
def slices(s, *args):
    position = 0
    for length in args:
        yield s[position:position + length]
        position += length

In [40]:
txtfile = open("may12pub.dat","r")
arr = []
for line in txtfile:
    arr.append(list(slices(line, 15,2,4,2,3,2,2,2,2,2,2,2,2,2,2,10,2,2,2,2,2,2,2,5,3,2,2,2,2,2,2,2,2,1,5,3,1,1,1,1,3,3,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,3,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,2,2,2,2,2,2,2,2,2,2,3,2,5,2,2,2,2,2,2,2,2,2,2,2,2,2,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,2,2,2,2,2,2,2,2,2,2,2,2,2,6,2,2,6,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,4,4,4,4,1,2,8,1,4,8,8,1,2,2,2,2,2,2,2,2,2,2,2,2,2,10,10,10,10,10,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,4,2,2,2,2,2,2,2,2,2,2,2,2,5,2,2,2,2,2,2,2,2,2,2,2,2,2,10,4,4,4,4,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,17,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,10)))

In [ ]:
disability = pd.DataFrame(arr)
disability.head(5)

In [ ]:
status = disability[84].astype(int)
status = np.where(status == -1, 1, status)
status = np.where(status == -2, 2, status)
status = np.where(status == -3, 3, status)
status = np.where(status == 2, 0, status)

unique, counts = np.unique(status, return_counts=True)
print(np.asarray((unique, counts)).T)

disability['status'] = status
disability = disability[disability['status'] != 3]

In [ ]:
new_disability = []
for i in range(len(disability)):
    new_disability.append(pd.to_numeric(disability.iloc[i],errors='coerce'))

In [ ]:
data = pd.DataFrame(new_disability)

In [ ]:
data.describe()
data = data.fillna(0)

In [ ]:
X = data.loc[:, (data.columns != 'status') & (data.columns != 84)& (data.columns != 0)& (data.columns != 1)& (data.columns != 2)]
len(X)

In [ ]:
y = data['status']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Make predictions on validation dataset
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
predictions = knn.predict(X_test)

print()
print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

In [ ]:
X_scaled = scale(data)

In [ ]:
pca = PCA(n_components=5)
pca.fit(X_scaled)
X_pca = pca.transform(X_scaled) @ pca.components_

In [ ]:
df = pd.DataFrame(X_pca)
df['target'] = pd.Series(y)

fig = px.scatter_3d(df, x=121, y=81, z=251,color='target')  # days of layoff and years of education 121,81,251
fig.show();

In [ ]:
df = pd.DataFrame(X_pca)
df['target'] = pd.Series(y)

fig = px.scatter_3d(df, x=121, y=100, z=251,color='target')  # days of layoff and years of education 
fig.show();

In [ ]:
df.to_csv(r'may2012data.csv')